In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split


In [2]:
data = pd.read_csv("/work/deepnote_exports/2024-12-06T01:05:10+00:00_3f3e.csv")
data.head()

,startingAirport,destinationAirport,elapsedDays,isBasicEconomy,isRefundable,isNonStop,baseFare,seatsRemaining,totalTravelDistance,segmentsArrivalAirportCode,...,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode,bookingClassCode,daysTillFlight,departureDayOfWeek,departureHour,arrivalDayOfWeek,arrivalHour
0,LAX,ORD,1,False,False,True,236.28,9,1745.0,ORD,...,Boeing 737-900,14640,1745,coach,W,27,5,23,6,6
1,LAX,ORD,0,False,False,True,305.12,6,1745.0,ORD,...,Boeing 757-300,14880,1745,coach,Q,3,1,10,1,16
2,LAX,ORD,0,False,False,True,322.79,5,1745.0,ORD,...,Boeing 737-800,14700,1745,coach,M,1,1,13,1,19
3,LAX,ORD,0,False,False,True,459.53,7,1745.0,ORD,...,Boeing 737-800,14640,1745,coach,H,8,0,7,0,13
4,LAX,DFW,0,False,False,True,252.09,7,1238.0,DFW,...,Airbus A321,10860,1238,coach,L,5,5,18,5,23


In [3]:
df_encoded = pd.get_dummies(data)
df_encoded = df_encoded.dropna()
df_encoded = df_encoded.astype('float64')
X = df_encoded.drop('baseFare', axis=1)
y = df_encoded['baseFare']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=142)

In [4]:
# Lasso with arbitrarily chosen alpha as 0.1 ----> corresponds to 0.80 r2 accuracy

from sklearn.linear_model import Lasso

lasso_model = Lasso(alpha=0.1)
lasso_model.fit(X_train, y_train)
y_pred = lasso_model.predict(X_test)

/root/venv/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.550e+07, tolerance: 3.628e+04
  model = cd_fast.enet_coordinate_descent(


In [5]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)
print(f"R² Score: {r2}")

R² Score: 0.8105480182933995


In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Fit LassoCV again on the scaled data
lasso_cv = LassoCV(cv=5, random_state=42).fit(X_train_scaled, y_train)
print("Optimal Alpha (scaled):", lasso_cv.alpha_)

Optimal Alpha (scaled): 0.05671289954355441


In [7]:
# Seems like CV-selected alpha achieves the same R² score as the original model with randomly choosing 0.1 as the alpha

lasso_model_CV_alpha = Lasso(alpha=0.05668193869844624)
lasso_model_CV_alpha.fit(X_train, y_train)
y_pred_CV_alpha = lasso_model_CV_alpha.predict(X_test)
r2_CV_alpha = r2_score(y_test, y_pred_CV_alpha)
print(f"R² Score: {r2_CV_alpha}")

/root/venv/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.270e+07, tolerance: 3.628e+04
  model = cd_fast.enet_coordinate_descent(
R² Score: 0.8143844526404496


In [8]:
features = X_train.columns
coefficients = lasso_model.coef_

In [9]:
feature_importance = pd.DataFrame({
    'Feature': features,
    'Coefficient': coefficients
})

# Filter dropped features (coefficients that are zero)
dropped_features = feature_importance[feature_importance['Coefficient'] == 0]
print("Dropped Features:")
print(dropped_features)

# Features retained
retained_features = feature_importance[feature_importance['Coefficient'] != 0]
print("\nRetained Features:")
print(retained_features)

Dropped Features:
                                              Feature  Coefficient
0                                         elapsedDays         -0.0
2                                        isRefundable          0.0
3                                           isNonStop          0.0
13                                startingAirport_LAX         -0.0
14                                startingAirport_SFO          0.0
15                             destinationAirport_ATL         -0.0
16                             destinationAirport_BOS         -0.0
17                             destinationAirport_CLT          0.0
18                             destinationAirport_DEN          0.0
20                             destinationAirport_DTW          0.0
21                             destinationAirport_EWR         -0.0
22                             destinationAirport_IAD          0.0
23                             destinationAirport_JFK         -0.0
24                             destinationAi

In [13]:
SSE = np.sum((y_test - y_pred_CV_alpha) ** 2)

# Calculate SST (using the mean of y_train)
SST = np.sum((y_test - np.mean(y_train)) ** 2)

# Compute OSR^2
OSR2 = 1 - SSE / SST

print(f"OSR²: {OSR2}")

OSR²: 0.8143970285952638


In [14]:
rmse = np.sqrt(np.mean((y_test - y_pred_CV_alpha) ** 2))
print("RMSE:", rmse)

RMSE: 62.94605030724557


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=36e46b6c-b77a-4f8c-a857-c3dc2e354e3d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>